In [ ]:
import pandas as pd 
import numpy as np 
from datetime import datetime

# Initial Data Inspection:

First I will load in the dataframe and inspect the first few rows to get a general feel of the data.

In [2]:
youtube_data = pd.read_csv('../raw_data/youtube_data.csv')
youtube_data.head()

,video_id,title,description,published_date,channel_id,channel_title,tags,category_id,view_count,like_count,comment_count,duration,thumbnail
0,gsJAlLOFBv0,TINY Tech That Actually Works,No description available,2025-05-02T17:37:10Z,UCMiJRAwDNSNzuYeN2uWa0pA,Mrwhosetheboss,"['tiny', 'tech', 'gadgets', 'small', 'miniature']",28,8962092.0,243350.0,515.0,PT57S,https://i.ytimg.com/vi/gsJAlLOFBv0/default.jpg
1,ypicIkaiViM,AI & future of workforce: Andrew Yang on how t...,"Andrew Yang, Forward Party co-chair and former...",2025-06-18T12:39:53Z,UCrp_UI8XtuYfpiqluWLD7Lw,CNBC Television,"['Squawk Box U.S.', 'CNBC', 'business news', '...",25,289626.0,3393.0,1240.0,PT7M50S,https://i.ytimg.com/vi/ypicIkaiViM/default.jpg
2,1Nef8LPO-jo,5 ILLEGAL gadgets that will get you ARRESTED,#shorts #technology \n\nI spend a LOT of time ...,2022-11-01T11:00:06Z,UCMiJRAwDNSNzuYeN2uWa0pA,Mrwhosetheboss,"['shorts', 'tech']",28,81372201.0,4178447.0,6378.0,PT47S,https://i.ytimg.com/vi/1Nef8LPO-jo/default.jpg
3,lCHqmzynO-s,Overrated vs. Underrated Tech,💬 Join my Discord server: https://discord.gg/g...,2024-07-08T18:04:31Z,UCPk2s5c4R_d-EUUNvFFODoA,Gohar Khan,"['thailand', 'surin', 'style', 'travel', 'day'...",27,21255964.0,909386.0,2681.0,PT31S,https://i.ytimg.com/vi/lCHqmzynO-s/default.jpg
4,7uFrtqSwYzM,APPLE Glass Revolutionizes AR Experience Forever!,Discover the revolutionary world of augmented ...,2024-12-22T16:49:00Z,UCxqG_E-68WAE0TWYfIopv6Q,Digifix,"['apple glasses price', 'apple glasses design'...",28,2790436.0,44278.0,1359.0,PT16S,https://i.ytimg.com/vi/7uFrtqSwYzM/default.jpg


Next I'll check the datatypes of each column as well as checking for any null values using *.info()*

In [3]:
youtube_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        600 non-null    object 
 1   title           600 non-null    object 
 2   description     600 non-null    object 
 3   published_date  600 non-null    object 
 4   channel_id      600 non-null    object 
 5   channel_title   600 non-null    object 
 6   tags            600 non-null    object 
 7   category_id     600 non-null    int64  
 8   view_count      600 non-null    float64
 9   like_count      600 non-null    float64
 10  comment_count   600 non-null    float64
 11  duration        600 non-null    object 
 12  thumbnail       600 non-null    object 
dtypes: float64(3), int64(1), object(9)
memory usage: 61.1+ KB


A couple of thing to note upon this early inspection of the DataFrame are:

* Publish date appears a string instead of a DateTime, converting to DateTime will make later analysis much easier
* Some videos potentially have a string containing 'No description' instead of a Null value.
* Catergory_id is a catergorical ID but with no reference point. Will need to find one to look at which catergory of videos perform best.
* Tags are in a list, finding effective tags might require some additional extraction.
* Duration is an arbretary code instead of an actual runtime. I am provided with the Video ID so maybe I could pull the runtimes with a helper function.

## Catergory ID:

If I want to do some analysis on different video catergories. I need a reference point for catergory_id else any comparison I do will be meaningless. To do this I'll run a request to the YouTube API and pull the info into a seperate DataFrame. To do this I wrote a helper function in the _catergory_id_fetcher.py_. This code extracted the catergory_ID's and wrote them into a new CSV.

In [4]:
category_id_data = pd.read_csv('../raw_data/catergory_ids.csv')
category_id_data.head()

,Unnamed,category_id,title
0,0,1,Film & Animation
1,1,2,Autos & Vehicles
2,2,10,Music
3,3,15,Pets & Animals
4,4,17,Sports


This DataFrame could do with a small amount of extra clean up. There is a typo in category_id that needs to be fixed so it will match with the other DataFram and there is a unecessary column that can be removed. Once that is done I'll save this as a piece of clean data ready for use.

In [5]:
category_id_data.rename(columns={'catergory_id' : 'category_id'}) # Correcting the column name typo to match the YouTube data.
category_id_data.drop(columns=['Unnamed'], inplace=True) # Removing the unnecessary column
category_id_data.to_csv('../clean_data/category_id_data_clean.csv',index=False)

## Video Duration:
The video duration is in a format that is not very useful for analysis. It is in the format of 'PT1H2M3S' which stands for 1 hour, 2 minutes and 3 seconds.
A better suited format would be an float64 format to make it easier to plot graphs and make meaningful comparissons. So I will convert each time into minutes and add them as a new column called duration_minutes.

In [6]:
def duration_to_minutes(duration):
    '''Converts Youtube_data duration from string format to minutes as an integer.'''
    duration = duration.replace('PT', '').replace('H', ' ').replace('M', ' ').replace('S', ' ') # Remove any non-numeric characters.
    time_parts = duration.split()
    # Split the string into parts based on spaces
    for part in time_parts:
        if part.isdigit():
            continue
        else:
            try:
                float(part) # If it can be converted to a float, it is a valid time part
            except ValueError:
                return duration # If it can't be converted, return the original string 
    if len(time_parts) == 3:
        hours = float(time_parts[0]) * 60
        minutes = float(time_parts[1])
        seconds = float(time_parts[2]) / 60
    elif len(time_parts) == 2:
        hours = 0
        minutes = float(time_parts[0])
        seconds = float(time_parts[1]) /60
    elif len(time_parts) == 1:
        hours = 0
        minutes = 0
        seconds = float(time_parts[0]) / 60
    else:
        hours = 0
        minutes = 0
        seconds = 0
    total_minutes = hours + minutes + seconds
    return round(total_minutes,2) # rounding to 2 decimal places for better readbility                
    


youtube_data['duration_minutes'] = youtube_data['duration'].apply(duration_to_minutes)
youtube_data.duration_minutes.unique()


array([0.95, 7.83, 0.78, 0.52, 0.27, 0.93, 0.02, 0.5, 7.28, 0.25, 14.85,
       0.4, 0.17, 0.28, 16.27, 33.5, 0.8, 37.87, 0.98, 0.47, 32.45, 4.15,
       9.17, 0.55, 0.75, 0.77, 31.43, 0.38, 0.42, 18.08, 0.97, 36.08,
       2.02, 18.12, 0.87, 26.03, 22.73, 0.22, 34.23, 0.43, 0.63, 1.02,
       0.3, 0.37, 0.45, 0.33, 0.35, 0.88, 0.2, 0.18, 0.1, 0.7, 0.53, 0.23,
       54.4, 0.32, 36.42, 0.73, 0.9, 39.9, 33.78, 3.65, 0.57, 1.78, 0.85,
       0.65, 5.17, 28.7, 0.82, 37.82, 1.15, 37.3, 0.13, 13.37, 1.12, 0.58,
       4.13, 48.8, 3.68, 18.32, 0.62, 34.55, 0.6, 16.07, 41.67, 0.83,
       0.15, 13.55, 1.93, 4.67, 15.13, 0.08, 1.63, 285.83, 19.8, 0.72,
       0.68, 4.47, 0.48, 19.22, 20.75, 0.12, 16.7, 16.87, 19.3, 14.67,
       27.8, 8.42, 25.8, 27.25, 5.72, 10.28, 20.17, 21.45, 9.63, 18.38,
       6.55, 0.92, 3.62, 46.35, 10.52, 0.67, 9.47, 21.9, 8.8, 7.57, 38.43,
       14.28, 33.12, 1.33, 14.1, 11.7, 25.38, 10.83, 1.52, 8.88, 16.47,
       6.3, 3.23, 3.57, 4.85, 11.5, 4.6, 59.35, 20.67, 10

As you can see, All but one value has been converted into minutes. One has been converted into 'P0D' and needs further exploration.

In [7]:
youtube_data.loc[youtube_data['duration_minutes'] == 'P0D']

,video_id,title,description,published_date,channel_id,channel_title,tags,category_id,view_count,like_count,comment_count,duration,thumbnail,duration_minutes
502,iEpJwprxDdk,Bloomberg Business News Live,Programming schedule (EST):\n12:00 AM - 5:00 A...,2024-10-18T14:53:08Z,UCIALMKvObZNtJ6AmdCLP7Lg,Bloomberg Television,"['finance', 'news', 'Bloomberg', 'business', '...",25,29732507.0,48267.0,0.0,P0D,https://i.ytimg.com/vi/iEpJwprxDdk/default_liv...,P0D


Upon inspection and some research, this video is showing P0D as its duration as it is a live feed on YouTube and it's length therefore ins't finite. To prevent this row from skewing any potential analysis I'll make I'm going to remove it and then I can change the type of the column to a Float which will better represent the data.

In [8]:
youtube_data.drop(502, inplace=True)
youtube_data.duration_minutes.unique()
youtube_data['duration_minutes'] = youtube_data['duration_minutes'].astype(float)

In [9]:
youtube_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 599 entries, 0 to 599
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   video_id          599 non-null    object 
 1   title             599 non-null    object 
 2   description       599 non-null    object 
 3   published_date    599 non-null    object 
 4   channel_id        599 non-null    object 
 5   channel_title     599 non-null    object 
 6   tags              599 non-null    object 
 7   category_id       599 non-null    int64  
 8   view_count        599 non-null    float64
 9   like_count        599 non-null    float64
 10  comment_count     599 non-null    float64
 11  duration          599 non-null    object 
 12  thumbnail         599 non-null    object 
 13  duration_minutes  599 non-null    float64
dtypes: float64(4), int64(1), object(9)
memory usage: 70.2+ KB


## Publish Date and Publish Time:

The last thing I feel as though I need to tidy before I call this dataset clean is the published_date column. Upon first inspection I thought It was a DateTime object but now I belive it is just a string. I can however convert it into a DateTime and I think for better analysis I will make three seperate columns. A published_date, A publish_day and a publish_time. This way I can find the most effective day of the week and the most effective time of the day to upload a video.

In [10]:
def convert_to_datetime(date_str):
    '''Converts a string in the format 'YYYY-MM-DDTHH:MM:SSZ' to a datetime object.'''
    date_str = date_str.replace('T',' ').replace('Z','') # Removing the T and Z characters
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

youtube_data['published_date'] = youtube_data['published_date'].apply(convert_to_datetime)
youtube_data.head(3)

,video_id,title,description,published_date,channel_id,channel_title,tags,category_id,view_count,like_count,comment_count,duration,thumbnail,duration_minutes
0,gsJAlLOFBv0,TINY Tech That Actually Works,No description available,2025-05-02 17:37:10,UCMiJRAwDNSNzuYeN2uWa0pA,Mrwhosetheboss,"['tiny', 'tech', 'gadgets', 'small', 'miniature']",28,8962092.0,243350.0,515.0,PT57S,https://i.ytimg.com/vi/gsJAlLOFBv0/default.jpg,0.95
1,ypicIkaiViM,AI & future of workforce: Andrew Yang on how t...,"Andrew Yang, Forward Party co-chair and former...",2025-06-18 12:39:53,UCrp_UI8XtuYfpiqluWLD7Lw,CNBC Television,"['Squawk Box U.S.', 'CNBC', 'business news', '...",25,289626.0,3393.0,1240.0,PT7M50S,https://i.ytimg.com/vi/ypicIkaiViM/default.jpg,7.83
2,1Nef8LPO-jo,5 ILLEGAL gadgets that will get you ARRESTED,#shorts #technology \n\nI spend a LOT of time ...,2022-11-01 11:00:06,UCMiJRAwDNSNzuYeN2uWa0pA,Mrwhosetheboss,"['shorts', 'tech']",28,81372201.0,4178447.0,6378.0,PT47S,https://i.ytimg.com/vi/1Nef8LPO-jo/default.jpg,0.78


### Now to add the published_day and published_time columns.

In [14]:
youtube_data['published_day'] =youtube_data['published_date'].dt.day_name()
youtube_data['published_time'] = youtube_data['published_date'].dt.time
youtube_data = youtube_data[['title', 'description', 'channel_title', 'category_id', 'published_date', 'published_day', 'published_time','duration_minutes', 'view_count', 'like_count', 'comment_count','tags','video_id']]
youtube_data.head(3)

,title,description,channel_title,category_id,published_date,published_day,published_time,duration_minutes,view_count,like_count,comment_count,tags,video_id
0,TINY Tech That Actually Works,No description available,Mrwhosetheboss,28,2025-05-02 17:37:10,Friday,17:37:10,0.95,8962092.0,243350.0,515.0,"['tiny', 'tech', 'gadgets', 'small', 'miniature']",gsJAlLOFBv0
1,AI & future of workforce: Andrew Yang on how t...,"Andrew Yang, Forward Party co-chair and former...",CNBC Television,25,2025-06-18 12:39:53,Wednesday,12:39:53,7.83,289626.0,3393.0,1240.0,"['Squawk Box U.S.', 'CNBC', 'business news', '...",ypicIkaiViM
2,5 ILLEGAL gadgets that will get you ARRESTED,#shorts #technology \n\nI spend a LOT of time ...,Mrwhosetheboss,28,2022-11-01 11:00:06,Tuesday,11:00:06,0.78,81372201.0,4178447.0,6378.0,"['shorts', 'tech']",1Nef8LPO-jo


That now looks like a clean dataframe ready for some analysis! Now I'll export it as a clean CSV.

In [15]:
youtube_data.to_csv('../clean_data/youtube_data_clean.csv', index=False)